In [1]:
import numpy as np
import pandas as pd
import matplotlib 
from matplotlib import pyplot as plt
%matplotlib inline
import time
import rebound
import xgboost as xgb
import sklearn
import os
from sklearn.model_selection import cross_val_score

from sklearn import metrics

/mnt/raid-cita/nhussain/venv-2.7.13/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Check if the feature importances change for short integrations vs longterm integrations


In [7]:
data_list = np.loadtxt("../csvs/LargeTimeScaleRange/Order.txt", dtype=str)

features = []
features += ['avg_iH1', 'avg_iH2']
#features += ['mindaOverRH', 'maxdaOverRH']
features += ['norm_std_a1', 'norm_max_a1', 'norm_std_window10_a1', 'norm_max_window10_a1']
features += ['norm_std_a2', 'norm_max_a2', 'norm_std_window10_a2', 'norm_max_window10_a2']
features += ['norm_std_a3', 'norm_max_a3', 'norm_std_window10_a3', 'norm_max_window10_a3']
features += ['avg_ecross1', 'std_ecross1', 'max_ecross1', 'min_ecross1']
features += ['avg_ecross2', 'std_ecross2', 'max_ecross2', 'min_ecross2']
features += ['avg_ecross3', 'std_ecross3', 'max_ecross3', 'min_ecross3']
features += ['norm_a1_slope', 'norm_a2_slope', 'norm_a3_slope']
features += ["max_e2", "min_e2", "std_e2", "avg_e2"]
features += ['Lyapunov_time']
#features += [""]

def make_splits(Ntrain, X, Y):
    
    Nrows = int(X.shape[0]*Ntrain)
    trainX = X.loc[:Nrows-1]
    testX = X.loc[Nrows:]
    trainY = Y[:Nrows]
    testY = Y[Nrows:]
    #print trainX.shape, trainY.shape, testX.shape, testY.shape
    return trainX, trainY, testX, testY

data_list

array(['sifOrbs=1000.0Nout=100window=10lyapwindow=10.csv',
       'sifOrbs=100.0Nout=10window=10lyapwindow=10.csv',
       'sifOrbs=10.0Nout=10window=10lyapwindow=10.csv',
       'sifOrbs=32000.0Nout=1000window=10lyapwindow=10.csv',
       'sifOrbs=3200.0Nout=100window=10lyapwindow=10.csv',
       'sifOrbs=320.0Nout=10window=10lyapwindow=10.csv',
       'sifOrbs=32.0Nout=10window=10lyapwindow=10.csv'],
      dtype='|S50')

In [3]:
def plot_feats_imps(model):
    feats = pd.Series(model.booster().get_fscore()).sort_values(ascending = False)
    total_imp = np.sum(feats)
    feats = feats/float(total_imp)
    feats.plot(kind = "bar", title = "Feature Importances", figsize = (12, 8))
    plt.ylabel("Normalized Importance")
    return feats

In [4]:
#{'x_min_child': 4.0, 'x_tree_colsample': 0.7452992521028268,
#'x_max_depth': 7.0, 'x_subsample': 0.938790987639708, 'x_learning_rate': 0.08584312332127479}

min_child_w = 4
n_est=100
max_d = 7
learning_r = 0.085
subsample = 0.94
weight = 2

model = xgb.XGBClassifier(min_child_weight=min_child_w, n_estimators=n_est, max_depth=max_d, learning_rate=learning_r, scale_pos_weight=weight, subsample=subsample)


In [22]:
#load one dataset to determine the number of features needed
#info_data = pd.read_csv("../csvs/LargeTimeScaleRange/sifOrbs=10.0Nout=10window=10lyapwindow=10.csv", index_col = 0)
#del info_data["runstring"]
#del info_data["t_final_short"]
#del info_data["Rel_Eerr_short"]
#del info_data["Rel_Eerr"]


#all_feats = info_data.columns.values.tolist()
#print all_feats
#all_feats = all_feats[:-3]#get ridd of walltime, stable and instabilit time
#print all_feats, len(all_feats)
#print all_feats
#score = np.zeros(len(all_feats))
all_feats_len = 120 #?
feat_scores = np.zeros((all_feats_len, data_list.shape[0])) #feats by total datasets
feat_scores_labels = [] #feats by total datasets
    
#['norm_std_a1'] NOT IN ALL DATASETS?
    
    
#model_scores = []
    
for i, name in enumerate(data_list):

    data = pd.read_csv("../csvs/LargeTimeScaleRange/" + name, index_col = 0)
    del data["runstring"]
    del data["t_final_short"]
    del data["Rel_Eerr_short"]
    del data["Rel_Eerr"]

    Y = data["Stable"]
    all_feats = data.columns.values.tolist()
    print len(all_feats)
    all_feats= all_feats[: -3]
    print len(all_feats)
    X = data[all_feats]
    trainX, trainY, testX, testY = make_splits(0.2, X, Y)
    model.fit(trainX, trainY)
    preds = model.predict(testX)
    feats = pd.Series(model.booster().get_fscore()) #this shpuld be in the order of features passed?
    print feats.shape
    feat_scores[:feats.shape[0],i] = feats.values
    feat_scores_labels.append(feats.index.values.tolist())
    #model_scores.append(ranked_feats)
    
    
    #create duplicate list and delete an element each time to see what is not consistent
    #important_feats = ranked_feats[:]
    #print important_feats, ranked_feats
    #for feat in ranked_feats:#reference list
    #    #print feat
    #    try:
    #        all_feats.remove(feat)
    #        important_feats.remove(feat)
    #    except Exception as e:
    #        #print e
    #        print "Not in both lists",  feat
    #print all_feats, important_feats
    
    
    
    
    
    
    
    
    
    
    #feat_scores[:,i] = feats.values
    score = metrics.average_precision_score(testY, preds)
    print score
    #break
    #important_features = plot_feats_imps(model)
#print data.columns.values, data.shape
#data.head()
print feat_scores_labels

115
112
(111,)
0.914184008748
115
112
(93,)
0.90866851522
115
112
(93,)
0.900852007768
115
112
(111,)
0.932475823938
115
112
(112,)
0.915554198959
115
112
(94,)
0.907888477304
115
112
(93,)
0.905119784965
[['Lyapunov_time', 'avg_a1', 'avg_a2', 'avg_a3', 'avg_e1', 'avg_e2', 'avg_e3', 'avg_eH1', 'avg_eH2', 'avg_eH3', 'avg_ecross1', 'avg_ecross2', 'avg_ecross3', 'avg_iH1', 'avg_iH2', 'avg_iH3', 'avg_inc1', 'avg_inc2', 'avg_inc3', 'max_a1', 'max_a2', 'max_a3', 'max_e1', 'max_e2', 'max_e3', 'max_eH1', 'max_eH2', 'max_eH3', 'max_ecross1', 'max_ecross2', 'max_ecross3', 'max_iH1', 'max_iH2', 'max_iH3', 'max_inc1', 'max_inc2', 'max_inc3', 'min_a1', 'min_a2', 'min_a3', 'min_e1', 'min_e2', 'min_e3', 'min_eH1', 'min_eH2', 'min_eH3', 'min_ecross1', 'min_ecross2', 'min_ecross3', 'min_iH1', 'min_iH2', 'min_iH3', 'min_inc1', 'min_inc2', 'min_inc3', 'norm_a1_slope', 'norm_a2_slope', 'norm_a3_slope', 'norm_max_a1', 'norm_max_a2', 'norm_max_a3', 'norm_max_e1', 'norm_max_e2', 'norm_max_e3', 'norm_max_inc1

In [ ]:
min_child_w = 10
n_est=500
max_d = 20
learning_r = 0.02
weight = 3
col_sample = 0.8
subsample = 0.95



model = xgb.XGBClassifier(min_child_weight=min_child_w, n_estimators=n_est, max_depth=max_d,
                          learning_rate=learning_r, scale_pos_weight=weight, subsample=subsample,
                         colsample_bytree = col_sample, seed = 0)
model.fit(trainX, trainY)
preds = model.predict(testX)
score = metrics.average_precision_score(testY, preds)
print score
important_feats = plot_feats_imps(model)

print "CROSS VALIDATION"
dtrain = xgb.DMatrix(X, label = Y)
params = model.get_xgb_params()
score = xgb.cv(params=params, dtrain=dtrain, nfold=5, metrics="auc", early_stopping_rounds=10, verbose_eval=True, stratified=True)
print score
print np.mean(score["test-auc-mean"])

In [ ]:
#See how it changes if you take the top 25?
top_feats = 25

X = data[important_feats[:top_feats].index]
trainX, trainY, testX, testY  = make_splits(0.2, X, Y)


model = xgb.XGBClassifier(min_child_weight=min_child_w, n_estimators=n_est, max_depth=max_d,
                          learning_rate=learning_r, scale_pos_weight=weight, subsample=subsample,
                         colsample_bytree = col_sample, seed = 0)
model.fit(trainX, trainY)
preds = model.predict(testX)
score = metrics.average_precision_score(testY, preds)
print score
important_feats = plot_feats_imps(model)

print "CROSS VALIDATION"
dtrain = xgb.DMatrix(X, label = Y)
params = model.get_xgb_params()
score = xgb.cv(params=params, dtrain=dtrain, nfold=5, metrics="auc", early_stopping_rounds=10, verbose_eval=True, stratified=True)
print score
print np.mean(score["test-auc-mean"])